# Poyecto Final IA, CLASIFICADOR DE ESTRELLAS
#Geuseppe Segrera
###Introducción
El presente proyecto corresponde a un clasificador de estrellas, en el cual se utilizaran 6 caracteristicas para clasificarlas:
* Clase espectral
* Color
* Temperatura
* Radio relativo
* Luminosidad relativa
* Magnitud absoluta

Junto con estas caracteristicas se pretende poder clasificar cada estrella de entre los 6 tipos que se tienen, los cuales son:
* Enana Roja (0)
* Enana Marron (1)
* Enana blanca (2)
* Secuencia principal (3)
* Super Gigante(4)
* Hiper Gigante (5)

Los metodos escogidos para realizar los casificadores son las maquinas de soporte vectorial y las redes neuronales artificiales.

Enlace del dataset: https://www.kaggle.com/brsdincer/star-type-classification/version/1


###Desarrollo del proyecto

In [1]:
#Se importan las librerias a usar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
from sklearn.decomposition import PCA, KernelPCA
from sklearn import svm
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
import tensorflow.keras as kr
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_curve,roc_auc_score


* Lectura y depuración de datos

In [3]:
#Se leen los datos
dataframe = pd.read_csv("Stars.csv",sep=';')

#Se observan los 10 primeros para saber como son
ten_samples=dataframe.head(10)
print(ten_samples)

#Contamos cuantas muestras hay en total
dataframe.count(axis=0)

   Temperature         L       R    A_M Color Spectral_Class  Type
0         3068  0.002400  0.1700  16.12   Red              M     0
1         3042  0.000500  0.1542  16.60   Red              M     0
2         2600  0.000300  0.1020  18.70   Red              M     0
3         2800  0.000200  0.1600  16.65   Red              M     0
4         1939  0.000138  0.1030  20.06   Red              M     0
5         2840  0.000650  0.1100  16.98   Red              M     0
6         2637  0.000730  0.1270  17.22   Red              M     0
7         2600  0.000400  0.0960  17.40   Red              M     0
8         2650  0.000690  0.1100  17.45   Red              M     0
9         2700  0.000180  0.1300  16.05   Red              M     0


Temperature       240
L                 240
R                 240
A_M               240
Color             240
Spectral_Class    240
Type              240
dtype: int64

In [4]:
#Visualizamos las caracteristicas cualitativas
print(dataframe.groupby('Spectral_Class').size())
print(dataframe.groupby('Color').size())

Spectral_Class
A     19
B     46
F     17
G      1
K      6
M    111
O     40
dtype: int64
Color
Blue                   56
Blue White             10
Blue white              4
Blue-White              1
Blue-white             26
Orange                  2
Orange-Red              1
Pale yellow orange      1
Red                   112
White                   7
White-Yellow            1
Whitish                 2
Yellowish               1
Yellowish White         3
white                   3
yellow-white            8
yellowish               2
dtype: int64


In [5]:
#Reemplazamos las cadenas de caracteres por datos cuantitativos
dataframe = dataframe.replace([ 'A', 'B', 'F', 'G', 'K', 'M', 'O'], [ 0, 1, 2, 3, 4, 5, 6])
dataframe = dataframe.replace(['Blue','Red'], [0,1])
dataframe = dataframe.replace([ 'Blue White', 'Blue white', 'Blue-White', 'Blue-white'], 2)
dataframe = dataframe.replace([ 'Orange', 'Orange-Red', 'Pale yellow orange'], 3)
dataframe = dataframe.replace([ 'White', 'Whitish', 'white'], 4)
dataframe = dataframe.replace([ 'White-Yellow', 'Yellowish White', 'yellow-white'], 5)
dataframe = dataframe.replace([ 'Yellowish', 'yellowish'], 6)

#Confirmamos el reemplazo
print(dataframe.groupby('Color').size())
print(dataframe.groupby('Spectral_Class').size())

#Creamos un nuevo archivo csv con los datos limpios
dataframe.to_csv('Stars_clean.csv')

Color
0     56
1    112
2     41
3      4
4     12
5     12
6      3
dtype: int64
Spectral_Class
0     19
1     46
2     17
3      1
4      6
5    111
6     40
dtype: int64


* Extracción caracteristicas y etiquetas, escalización, división entre train y test, y verificación de uso PCA 

In [ ]:
#Extraemos las caractersiticas del dataset
features = dataframe[['Temperature', 'L', 'R', 'A_M','Color','Spectral_Class']].values
#Extraemos las etiquetas
labels = dataframe['Type'].values

#Escalizamos las caracteristicas
scaler = StandardScaler()
scaler.fit(features)# el fit de los datos
features = scaler.transform(features)

#Se verifica si es prudente realizar PCA
pca = decomposition.PCA(n_components=5,whiten=True,svd_solver='arpack')
pca.fit(features)
features = pca.transform(features)
var_expl=pca.explained_variance_ratio_
print("Pesos de PCA:",var_expl)
add=var_expl[0]+var_expl[1]+var_expl[2]+var_expl[3]+var_expl[4]
print("Varianza explicada porlas cinco primeras componenetes:",add)

#Dividimos en conjunto de test y train con un 75% para el conjunto de train y 25% para test
X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state=0,train_size=0.75)

#Mostramos la cantidad de muestras para el conjunto de train y test
print(X_train.shape)
print(X_test.shape)

Pesos de PCA: [0.4259625  0.24111583 0.17319635 0.07517904 0.04529431]
Varianza explicada porlas cinco primeras componenetes: 0.9607480380883548
(180, 5)
(60, 5)


* Se busca el mejor clasificador utilizando Maquinas de soporte vectorial

In [ ]:
#Se crean las listas para guardar lo hiperparámetros a iterar
kernels=['linear', 'poly', 'rbf', 'sigmoid']
decision=['ovo','ovr']
lista_hiper_param_MSV=[]
lista_metrics_MSV=[]
lista_metrics_mean_MSV=[]
#Se iteran los hiperparámetros
for iter_C in np.arange(1,6,0.1):#Se itera el coeficiente regularización
  for iter_shape in np.arange(0,1,1):#Se itera la desición multiclase
    for iter_kernel in np.arange(0,3,1):#Se itera el tipo de kernel
      #Se configura el método
      msv = svm.SVC(kernel=kernels[iter_kernel],C=(0.001*(10**iter_C)),decision_function_shape=decision[iter_shape],probability=True)
      
      #Se ajusta el método a los datos
      msv.fit(X_train, y_train)

      #Se predicen las etiquetas y sus probabilidades para el conjunto de entrenamiento
      y_train_predicted_MSV=msv.predict(X_train)
      y_train_scores_MSV=msv.predict_proba(X_train)

      #Se obtienen los valores de las metricas para el conjunto de entrenamiento
      MCC_MSV_train = matthews_corrcoef(y_train, y_train_predicted_MSV)
      F1_MSV_train = f1_score(y_train, y_train_predicted_MSV,average='macro')
      roc_auc_MSV_train=roc_auc_score(y_train, y_train_scores_MSV,multi_class=decision[iter_shape])

      #Se predicen las etiquetas y sus probabilidades para el conjunto de validación
      y_test_predicted_MSV=msv.predict(X_test)
      y_test_scores_MSV=msv.predict_proba(X_test)

      #Se obtienen los valores de las metricas para el conjunto de validación
      MCC_MSV_test = matthews_corrcoef(y_test, y_test_predicted_MSV)
      F1_MSV_test = f1_score(y_test, y_test_predicted_MSV,average='macro')
      roc_auc_MSV_test=roc_auc_score(y_test, y_test_scores_MSV,multi_class=decision[iter_shape])

      #Se guardan los valores de hiperparametros y metricas, tanto en el conjunto de train como en el de test, para cada iteración
      hiper_param_MSV=(iter_C,iter_shape,iter_kernel)
      metrics_MSV=(MCC_MSV_train, F1_MSV_train,roc_auc_MSV_train,MCC_MSV_test, F1_MSV_test,roc_auc_MSV_test)
      lista_metrics_mean_MSV.append(np.mean(metrics_MSV))
      lista_metrics_MSV.append(metrics_MSV)
      lista_hiper_param_MSV.append(hiper_param_MSV)

* Se encuentran los hiperparametros correspondientes al mejor clasificador por medio de maquinas de soporte vectorial 

In [ ]:
#Se imprimen los hiperparámetros para los cuales el promedio de las métricas fué el mayor
best_hiper_param_MSV=lista_hiper_param_MSV[np.argmax(lista_metrics_mean_MSV)]
print(best_hiper_param_MSV)
#Se imprime el promedio más alto de las métricas
print(lista_metrics_mean_MSV[np.argmax(lista_metrics_mean_MSV)])
#Se imprimen los valores de cada métrica para las cuales dió el promedio mas alto
print(lista_metrics_MSV[np.argmax(lista_metrics_mean_MSV)])

(3.500000000000002, 0, 0)
1.0
(1.0, 1.0, 1.0, 1.0, 1.0, 1.0)


* Se comprueba el funcionamiento del clasificador usando los hiperparámetros encontrados anteriormente

In [ ]:
#Se configura el clasificador
msv = svm.SVC(kernel=kernels[best_hiper_param_MSV[2]],C=(0.001*(10**best_hiper_param_MSV[0])),decision_function_shape=decision[best_hiper_param_MSV[1]],probability=True)

#Se ajusta el método a los datos
msv.fit(X_train, y_train)

#Se predicen las etiquetas y sus probabilidades para el conjunto de entrenamiento
y_train_predicted_MSV=msv.predict(X_train)
y_train_scores_MSV=msv.predict_proba(X_train)

#Se obtienen las métricas para el conjunto de entrenamiento
MCC_MSV_train = matthews_corrcoef(y_train, y_train_predicted_MSV)
F1_MSV_train = f1_score(y_train, y_train_predicted_MSV,average='macro')
roc_auc_MSV_train=roc_auc_score(y_train, y_train_scores_MSV,multi_class=decision[best_hiper_param_MSV[1]])

#Se predicen las etiquetas y sus probabilidades para el conjunto de validación
y_test_predicted_MSV=msv.predict(X_test)
y_test_scores_MSV=msv.predict_proba(X_test)

#Se obtienen las métricas para el conjunto de validación
MCC_MSV_test = matthews_corrcoef(y_test, y_test_predicted_MSV)
F1_MSV_test = f1_score(y_test, y_test_predicted_MSV,average='macro')
roc_auc_MSV_test=roc_auc_score(y_test, y_test_scores_MSV,multi_class=decision[0])

#Se imprimen las métricas
print("MCC_MSV_train:",MCC_MSV_train)
print("F1_MSV_train:",F1_MSV_train)
print("AUC_MSV_train:",roc_auc_MSV_train)
print("MCC_MSV_test:",MCC_MSV_test)
print("F1_MSV_test:",F1_MSV_test)
print("AUC_MSV_test:",roc_auc_MSV_test)

MCC_MSV_train: 1.0
F1_MSV_train: 1.0
AUC_MSV_train: 1.0
MCC_MSV_test: 1.0
F1_MSV_test: 1.0
AUC_MSV_test: 1.0


* Se busca el mejor clasificador utilizando redes neuronales

In [ ]:
#Se crean las listas para guardar lo hiperparametros a iterar
funtion=['identity', 'logistic', 'tanh', 'relu']
lista_hiper_param_ANN=[]
lista_metrics_ANN=[]
lista_metrics_mean_ANN=[]
#Se iteran los hiperparametros
for iter_h_layers in np.arange(1,20,1):#Se itera el la cantidad de capas ocultas
  for iter_neurons in np.arange(1,20,1):#Se itera la cantidad de neuronas
    for iter_funtion in np.arange(0,3,1):#Se itera las función de decisión

      #Configuramos el método
      ANN = MLPClassifier(hidden_layer_sizes=(iter_h_layers, iter_neurons),activation=funtion[iter_funtion],random_state=1, max_iter=500,verbose=False,learning_rate='adaptive', learning_rate_init=0.01)
      
      #Entrenamos el método
      ANN.fit(X_train, y_train)

      #Se predicen las etiquetas y sus probabilidades para el conjunto de entrenamiento
      y_train_predicted_ANN=ANN.predict(X_train)
      y_train_scores_ANN=ANN.predict_proba(X_train)

      #Se obtienen las metricas para el conjunto de entrenamiento
      MCC_ANN_train = matthews_corrcoef(y_train, y_train_predicted_ANN)
      F1_ANN_train = f1_score(y_train, y_train_predicted_ANN,average='macro')
      roc_auc_ANN_train = roc_auc_score(y_train, y_train_scores_ANN,multi_class='ovo')

      #Se predicen las etiquetas y sus probabilidades para el conjunto de validación
      y_test_predicted_ANN=ANN.predict(X_test)
      y_test_scores_ANN=ANN.predict_proba(X_test)

      #Se obtienen las metricas para el conjunto de validación
      MCC_ANN_test = matthews_corrcoef(y_test, y_test_predicted_ANN)
      F1_ANN_test = f1_score(y_test, y_test_predicted_ANN,average='macro')
      roc_auc_ANN_test = roc_auc_score(y_test, y_test_scores_ANN,multi_class='ovo')

      #Se guardan los valores de hiperparametros y metricas, tanto en el conjunto de train como en el de test, para cada iteración
      hiper_param_ANN=(iter_h_layers,iter_neurons,iter_funtion)
      metrics_ANN=(MCC_ANN_train, F1_ANN_train,roc_auc_ANN_train,MCC_ANN_test, F1_ANN_test,roc_auc_ANN_test)
      lista_metrics_mean_ANN.append(np.mean(metrics_ANN))
      lista_metrics_ANN.append(metrics_ANN)
      lista_hiper_param_ANN.append(hiper_param_ANN)


* Se encuentran los hiperparametros correspondientes al mejor clasificador por medio de redes neuronales 

In [ ]:
#Se imprimen los hiperparámetros para los cuales el promedio de las métricas fué el mayor
best_hiper_param_ANN=lista_hiper_param_ANN[np.argmax(lista_metrics_mean_ANN)]
print(best_hiper_param_ANN)
#Se imprime el promedio más alto de las métricas
print(lista_metrics_mean_ANN[np.argmax(lista_metrics_mean_ANN)])
#Se imprimen los valores de cada métrica para las cuales dió el promedio mas alto
print(lista_metrics_ANN[np.argmax(lista_metrics_mean_ANN)])

(2, 8, 0)
1.0
(1.0, 1.0, 1.0, 1.0, 1.0, 1.0)


* Se comprueba el funcionamiento del clasificador usando los hiperparámetros encontrados anteriormente

In [ ]:
#Configuramos el método
ANN = MLPClassifier(hidden_layer_sizes=(best_hiper_param_ANN[0], best_hiper_param_ANN[1]),activation=funtion[best_hiper_param_ANN[2]],random_state=1, max_iter=500,verbose=False,learning_rate='adaptive', learning_rate_init=0.01)

#Entrenamos el método
ANN.fit(X_train, y_train)

#Se predicen las etiquetas y sus probabilidades para el conjunto de entrenamiento
y_train_predicted_ANN=ANN.predict(X_train)
y_train_scores_ANN=ANN.predict_proba(X_train)

#Se obtienen las métricas para el conjunto de entrenamiento
MCC_ANN_train = matthews_corrcoef(y_train, y_train_predicted_ANN)
F1_ANN_train = f1_score(y_train, y_train_predicted_ANN,average='macro')
roc_auc_ANN_train = roc_auc_score(y_train, y_train_scores_ANN,multi_class='ovo')

#Se predicen las etiquetas y sus probabilidades para el conjunto de validación
y_test_predicted_ANN=ANN.predict(X_test)
y_test_scores_ANN=ANN.predict_proba(X_test)

#Se obtienen las métricas para el conjunto de validación
MCC_ANN_test = matthews_corrcoef(y_test, y_test_predicted_ANN)
F1_ANN_test = f1_score(y_test, y_test_predicted_ANN,average='macro')
roc_auc_ANN_test = roc_auc_score(y_test, y_test_scores_ANN,multi_class='ovo')

#Se imprimen las métricas
print("MCC_ANN_train:",MCC_ANN_train)
print("F1_ANN_train:",F1_ANN_train)
print("AUC_ANN_train:",roc_auc_ANN_train)
print("MCC_ANN_test:",MCC_ANN_test)
print("F1_ANN_test:",F1_ANN_test)
print("AUC_ANN_test:",roc_auc_ANN_test)

MCC_ANN_train: 1.0
F1_ANN_train: 1.0
AUC_ANN_train: 1.0
MCC_ANN_test: 1.0
F1_ANN_test: 1.0
AUC_ANN_test: 1.0


###Conclusiones
* Se encontró que ambos métodos producen clasificadores con una puntuación excelente en las métricas utilizadas, la selección entre estos recaería entonces en la velocidad y consumo de recursos de cada uno.
* Se encontró por medio de una busqueda por fuerza bruta los mejores hiperparámetros para cada método, estos dieron las metricas tanto en el conjunto de validación como en el de entrenamiento todas con un valor de 1 lo que quiere decir que se clasifican correctamente todas las muestras.
* No se hizo uso de PCA ya que las 5 primeras componentes principales de los datos no explican la varianza lo suficiente, dando un valor de 96.07% lo que no llega al valor esperado que es de por lo menos mayor al 97%. Por esto no se llegaba a un beneficio utilizando PCA.
* Se encontró que los métodos lineales separaban de una manera excelente los datos, ya que al hacer la iteración de hiperparámetros estos dieron las mejores métricas.
* Para decidir cual era el mejor conjunto de datos se tomo en cuenta las métricas evaluadas para el conjunto de entrenamiento y el de evaluación para así no sobre ajustar el método a ninguno de los dos conjuntos y encontrar realmente cual producia mejores resultados, esto se hizo almacenando el promedio de todas estas métricas y encontrando su valor máximo de entre todas las iteraciones hechas.
